## TFlite Prediction For Bounding Box

In [1]:
!pip install -U --no-cache-dir tensorflow==2.5.0
!pip install -U numpy==1.18.5
!pip install -U PILLOW==7.2.0
!pip install -U opencv-python==4.5.1.48
!pip install -U tf-slim

You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


  Using cached numpy-1.18.5-cp38-cp38-win_amd64.whl (12.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.18.5 which is incompatible.
datature-hub 0.2.0 requires tensorflow==2.3.0, but you have tensorflow 2.5.0 which is incompatible.
You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


  Using cached Pillow-7.2.0-cp38-cp38-win_amd64.whl (2.1 MB)
  Attempting uninstall: PILLOW
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datature-hub 0.2.0 requires Pillow~=8.2, but you have pillow 7.2.0 which is incompatible.
datature-hub 0.2.0 requires tensorflow==2.3.0, but you have tensorflow 2.5.0 which is incompatible.
You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


### Import all dependencies 

In [8]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" ## Uncomment to use GPU
import cv2
import numpy as np
import tensorflow as tf

from PIL import Image
from matplotlib.pyplot import imshow

## Prepare utility functions

In [4]:
def load_label_map(label_map_path):
    """Reads label map in the format of .pbtxt and parse into dictionary

    Args:
      label_map_path: the file path to the label_map

    Returns:
      dictionary with the format of {label_index: {'id': label_index, 'name': label_name}}
    """
    label_map = {}

    with open(label_map_path, "r") as label_file:
        for line in label_file:
            if "id" in line:
                label_index = int(line.split(":")[-1])
                label_name = next(label_file).split(":")[-1].strip().strip("'")
                label_map[label_index] = {
                    "id": label_index,
                    "name": label_name,
                }

    return label_map

## Prediction process

In [5]:
INPUT = "./input"
OUTPUT = "./output"
SIZE = [640,640]
THRESHOLD = 0.7
MODEL = "./tf.lite"
LABEL = "./label_map.pbtxt"

### Load model into interpreter

In [9]:
interpreter = tf.lite.Interpreter(model_path=MODEL)

### Visualize details of input & output layers 

In [10]:
## Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

## Print output details.
for idx, each_out in enumerate(output_details):
    print("{}: {}\n".format(idx,each_out))

0: {'name': 'StatefulPartitionedCall:5', 'index': 409, 'shape': array([1]), 'shape_signature': array([1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

1: {'name': 'StatefulPartitionedCall:1', 'index': 497, 'shape': array([1, 1, 1]), 'shape_signature': array([ 1, -1, -1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

2: {'name': 'StatefulPartitionedCall:2', 'index': 479, 'shape': array([1, 1]), 'shape_signature': array([ 1, -1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

3:

### Redefine expected image size of input layer 

In [11]:
## Current size is set to accept 320 by 320 image
interpreter.resize_tensor_input(input_details[0]['index'], (1, SIZE[0], SIZE[1], 3))
interpreter.allocate_tensors()

### Run prediction on each image

In [15]:
for each_image in os.listdir(INPUT):
        print("processing "+each_image)
        ### Load image into numpy array and resize accordingly
        img = cv2.imread(INPUT+'/'+each_image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        origi_shape = img.shape
        new_img = cv2.resize(img, (SIZE[0], SIZE[1]))
        
        ### Execute prediction

        interpreter.set_tensor(input_details[0]['index'], [new_img.astype(np.uint8)])
        interpreter.invoke()
        

        ### Extract data and filter against desired threshold
        for each_layer in output_details:
            if each_layer["name"] == "StatefulPartitionedCall:4":
                scores = np.squeeze(interpreter.get_tensor(each_layer['index']))
                
            if each_layer["name"] == "StatefulPartitionedCall:1":
                bboxes = np.squeeze(interpreter.get_tensor(each_layer['index']))
                
            if each_layer["name"] == "StatefulPartitionedCall:2":
                classes = np.squeeze(interpreter.get_tensor(each_layer['index']))

        ## Current desired confidence threshold is set at 0.7
        indexes = np.where(scores > THRESHOLD)

        ## Filter all prediction data above threshold
        scores = scores[indexes]
        bboxes = bboxes[indexes]
        classes = classes[indexes]


        ### Load label map and color map
        category_index = load_label_map(LABEL)

        color_map = {}
        for each_class in range(len(category_index)):
            color_map[each_class] = [
                int(i) for i in np.random.choice(range(256), size=3)
            ]

        ### Draw mask, bounding box, label onto the original image
        image_origi = np.array(img)

        if len(bboxes) != 0:
            for idx, each_bbox in enumerate(bboxes):
                color = color_map.get(classes[idx] - 1)

                ## Draw bounding box
                cv2.rectangle(
                    image_origi,
                    (
                        int(each_bbox[1] * origi_shape[1]),
                        int(each_bbox[0] * origi_shape[0]),
                    ),
                    (
                        int(each_bbox[3] * origi_shape[1]),
                        int(each_bbox[2] * origi_shape[0]),
                    ),
                    color,
                    2,
                )

                ## Draw label background
                cv2.rectangle(
                    image_origi,
                    (
                        int(each_bbox[1] * origi_shape[1]),
                        int(each_bbox[2] * origi_shape[0]),
                    ),
                    (
                        int(each_bbox[3] * origi_shape[1]),
                        int(each_bbox[2] * origi_shape[0] + 15),
                    ),
                    color,
                    -1,
                )

                ## Insert label class & score
                cv2.putText(
                    image_origi,
                    "Class: {}, Score: {}".format(
                        str(category_index[classes[idx]]["name"]),
                        str(round(scores[idx], 2)),
                    ),
                    (
                        int(each_bbox[1] * origi_shape[1]),
                        int(each_bbox[2] * origi_shape[0] + 10),
                    ),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.3,
                    (0, 0, 0),
                    1,
                    cv2.LINE_AA,
                )

        ## Save predicted image
        filename = os.path.basename(each_image)
        image_predict = Image.fromarray(image_origi)
        image_predict.save(os.path.join(OUTPUT, filename))

        print(
            "Saving predicted images to {}...".format(
                os.path.join(OUTPUT, filename)
            )
        )

processing boat161.png
Saving predicted images to ./output\boat161.png...
processing boat75.png
Saving predicted images to ./output\boat75.png...
processing boat94.png
Saving predicted images to ./output\boat94.png...
